In [1]:
# Import necessary module
from sqlalchemy import create_engine

# Create engine: engine
engine=create_engine('sqlite:///sqlite_db_pythonsqlite.db')

# Save the table names to a list: table_names
table_names=engine.table_names()

# Print the table names to the shell
print(table_names)


['Bookings', 'Facilities', 'Members']


In [2]:
# import packages
import pandas as pd
import sqlite3 

# connect database and create cursor
conn = sqlite3.connect('sqlite_db_pythonsqlite.db')
cur = conn.cursor()

# executing the query
cur.execute("""SELECT * FROM Facilities LIMIT 5;""")

# creating a dataframe object
df = pd.DataFrame(cur.fetchall())

# find database attributes from cursor.description to use as column names
cur.description

# creating dataframe columns
df.columns = [x[0] for x in cur.description]
df.head()

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000


In [3]:
# Part-2 Question 10: Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue. Remember
# that there's a different cost for guests and members!

# Import packages
#from flask_sqlalchemy 
from sqlalchemy import create_engine
import pandas as pd

# Create engine: engine
#engine = create_engine('sqlite:///Chinook.sqlite')sqlite_db_pythonsqlite.db
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')

# Open engine connection: con
con=engine.connect()

# Perform query: rs 
rs = con.execute(""" SELECT sub.name AS facility, sum(sub.cost) as tot_rev
       from (select f.name as name,
             case when B.memid=0 
                  then B.slots*f.guestcost
                  else B.slots*f.membercost
                  end as cost,
                  B.memid AS bmemid
              FROM Facilities as f
              join Bookings as B
              on f.facid=B.facid
              WHERE (B.memid != 0 AND f.membercost * B.slots)
              OR (B.memid = 0 AND f.guestcost * B.slots )
              order by cost desc) as sub
              join Members as M 
              on sub.bmemid=M.memid
              group by sub.name 
              having tot_rev < 1000 
              order by tot_rev; """)#SELECT * FROM `Facilities` WHERE facid in (1,5)")

# Save results of the query to DataFrame: df
df = pd.DataFrame(rs.fetchall())

# Close connection
con.close()

# Print head of DataFrame df
print(df.head())


               0    1
0   Table Tennis  180
1  Snooker Table  240
2     Pool Table  270


In [4]:
# Part-2 Question 11: Produce a report of members and who recommended them in alphabetic surname,firstname order 
# Import packages
#from flask_sqlalchemy 
from sqlalchemy import create_engine
import pandas as pd

# Create engine: engine
#engine = create_engine('sqlite:///Chinook.sqlite')sqlite_db_pythonsqlite.db
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')

# Open engine connection: con
con=engine.connect()

# Perform query: rs
rs = con.execute(""" SELECT m.firstname || ' ' || m.surname AS member_name,  r.firstname || ' ' || r.surname AS recommender_name 
FROM Members AS m
inner JOIN Members AS r 
ON (m.recommendedby = r.memid)
ORDER BY m.surname, m.firstname desc; """)#SELECT * FROM `Facilities` WHERE facid in (1,5)")

# Save results of the query to DataFrame: df
df = pd.DataFrame(rs.fetchall())

# Close connection
con.close()

# Print head of DataFrame df
print(df.head())


                0                1
0  Florence Bader  Ponder Stibbons
1   Timothy Baker   Jemima Farrell
2      Anne Baker  Ponder Stibbons
3      Tim Boothe       Tim Rownam
4  Gerald Butters     Darren Smith


In [5]:
# Part-2 Question 12: Find the facilities with their usage by member, but not guests
# Import packages
#from flask_sqlalchemy 
from sqlalchemy import create_engine
import pandas as pd

# Create engine: engine
#engine = create_engine('sqlite:///Chinook.sqlite')sqlite_db_pythonsqlite.db
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')

# Open engine connection: con
con=engine.connect()

# Perform query: rs
rs = con.execute(""" SELECT f.name AS facility,  M.firstname || ' ' || M.surname AS name,
case when B.memid!=0 then count(B.facid)  end as usagebymember

FROM Facilities as f
join Bookings as B
on f.facid=B.facid
join Members as M 
on B.memid=M.memid

group by f.name, name
order by usagebymember desc;
 """)#SELECT * FROM `Facilities` WHERE facid in (1,5)")

# Save results of the query to DataFrame: df
df = pd.DataFrame(rs.fetchall())

# Close connection
con.close()

# Print head of DataFrame df
print(df.head())


                0             1      2
0      Pool Table  Darren Smith  836.0
1  Massage Room 1  Darren Smith  629.0
2   Snooker Table  Darren Smith  444.0
3  Tennis Court 1   Tracy Smith  408.0
4    Table Tennis  Darren Smith  403.0


In [6]:
# Part-2 Question 13: Find the facilities usage by month, but not guests
# Import packages
#from flask_sqlalchemy 
from sqlalchemy import create_engine
import pandas as pd

# Create engine: engine
#engine = create_engine('sqlite:///Chinook.sqlite')sqlite_db_pythonsqlite.db
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')

# Open engine connection: con
con=engine.connect()

# Perform query: rs
rs = con.execute(""" 
SELECT f.name AS facility, M.firstname || ' ' || M.surname AS name,
case when B.memid=0 then count(B.facid)  end as usagebyguest

FROM Facilities as f
join Bookings as B
on f.facid=B.facid
join Members as M 
on B.memid=M.memid

group by name
order by usagebyguest desc;
 """)#SELECT * FROM `Facilities` WHERE facid in (1,5)")

# Save results of the query to DataFrame: df
df = pd.DataFrame(rs.fetchall())

# Close connection
con.close()

# Print head of DataFrame df
print(df.head())


                0             1      2
0    Squash Court   GUEST GUEST  440.0
1  Tennis Court 2   GUEST GUEST  389.0
2  Massage Room 2   GUEST GUEST  111.0
3  Tennis Court 1   Tracy Smith    NaN
4    Table Tennis  Darren Smith    NaN
